# RBF Regression on Boston hosting Dataset
### Charis Filis
### Academic-id : 9449 

Add needed dependencies

In [1]:
import tensorflow as tf
from tensorflow.keras import layers,Model
from tensorflow.keras import initializers
from tensorflow.keras import backend as K

from tensorflow.keras.datasets import boston_housing
# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.metrics import pairwise
from sklearn.cluster import KMeans
import numpy as np

In [ ]:
(x_train, y_train),(x_test, y_test) = boston_housing.load_data()

print("Number of original training examples:", len(x_train))
print("Number of original test examples:", len(x_test))
print("Training data shape:",x_train.shape)
print("Training y train shape", y_train.shape)
print("Testing data shape:",x_test.shape)

print(x_train[1], y_train[1])
# get per-feature statistics (mean,standard deviation ) from the training set in order to make normalization


In [1]:
# custom Kmeans algorithm that returns cluster centers and standart deviation of each cluster
"""
1D-Input Clustering
A := numpy array MX1 - Data
k := integer - Number of Clusters
"""
def kmeans_custom(A,k):
    
    # get Number of Features
    numFeatures = A.shape[1]
    # random initialization of centroids
    clusters_c = np.random.choice(np.squeeze(A),size=k)
    # kmeans is a greedy algorithm so we need a converge flag to stop him from running infinately
    converge_flag = False
    # initialize standart deviation buffer
    stds_c = np.zeros(k)
    
    while not converge_flag:
        """
            Compute distances of each cluster center to each point 
            of the dataset
        """
        

SyntaxError: incomplete input (574062755.py, line 19)